<a href="https://colab.research.google.com/github/yuki-tamaribuchi/ml_code_kata/blob/master/kaggle_house_prices_data_processing_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Load

In [1]:
!pip install category_encoders

In [2]:
!mkdir /root/.kaggle
!cp /content/drive/MyDrive/kaggle/kaggle.json /root/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [3]:
!kaggle competitions download -c house-prices-advanced-regression-techniques

data_description.txt: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv: Skipping, found more recently modified local copy (use --force to force download)
train.csv: Skipping, found more recently modified local copy (use --force to force download)
test.csv: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
!cat data_description.txt

MSSubClass: Identifies the type of dwelling involved in the sale.	

        20	1-STORY 1946 & NEWER ALL STYLES
        30	1-STORY 1945 & OLDER
        40	1-STORY W/FINISHED ATTIC ALL AGES
        45	1-1/2 STORY - UNFINISHED ALL AGES
        50	1-1/2 STORY FINISHED ALL AGES
        60	2-STORY 1946 & NEWER
        70	2-STORY 1945 & OLDER
        75	2-1/2 STORY ALL AGES
        80	SPLIT OR MULTI-LEVEL
        85	SPLIT FOYER
        90	DUPLEX - ALL STYLES AND AGES
       120	1-STORY PUD (Planned Unit Development) - 1946 & NEWER
       150	1-1/2 STORY PUD - ALL AGES
       160	2-STORY PUD - 1946 & NEWER
       180	PUD - MULTILEVEL - INCL SPLIT LEV/FOYER
       190	2 FAMILY CONVERSION - ALL STYLES AND AGES

MSZoning: Identifies the general zoning classification of the sale.
		
       A	Agriculture
       C	Commercial
       FV	Floating Village Residential
       I	Industrial
       RH	Residential High Density
       RL	Residential Low Density
       RP	Residential Low Density Park 
       RM

In [5]:
import pandas as pd
import numpy as np
import missingno as msno
import seaborn as sns

pd.options.display.max_columns=None
np.set_printoptions(threshold=np.inf)

In [6]:
df=pd.read_csv('train.csv')

In [7]:
train_df=df

train_df=train_df.drop(['Id'],axis=1)

train_df['MiscFeature']=train_df['MiscFeature'].fillna('none')
train_df['FireplaceQu']=train_df['FireplaceQu'].fillna('none')
train_df['Fence']=train_df['Fence'].fillna('none')
train_df['PoolQC']=train_df['PoolQC'].fillna('none')
train_df['Alley']=train_df['Alley'].fillna('none')
train_df['BsmtQual']=train_df['BsmtQual'].fillna('none')
train_df['BsmtCond']=train_df['BsmtCond'].fillna('none')
train_df['BsmtExposure']=train_df['BsmtExposure'].fillna('none')
train_df['BsmtFinType1']=train_df['BsmtFinType1'].fillna('none')
train_df['BsmtFinType2']=train_df['BsmtFinType2'].fillna('none')
train_df['GarageType']=train_df['GarageType'].fillna('none')
train_df['GarageYrBlt']=train_df['GarageYrBlt'].fillna('none')
train_df['GarageFinish']=train_df['GarageFinish'].fillna('none')
train_df['GarageQual']=train_df['GarageQual'].fillna('none')
train_df['GarageCond']=train_df['GarageCond'].fillna('none')

In [8]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1201 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   object 
 5   Alley          1460 non-null   object 
 6   LotShape       1460 non-null   object 
 7   LandContour    1460 non-null   object 
 8   Utilities      1460 non-null   object 
 9   LotConfig      1460 non-null   object 
 10  LandSlope      1460 non-null   object 
 11  Neighborhood   1460 non-null   object 
 12  Condition1     1460 non-null   object 
 13  Condition2     1460 non-null   object 
 14  BldgType       1460 non-null   object 
 15  HouseStyle     1460 non-null   object 
 16  OverallQual    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  YearBuil

In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(train_df.drop(['SalePrice'],axis=1),train_df['SalePrice'],random_state=0)

In [10]:
from category_encoders import TargetEncoder

num_cols=X_train._get_numeric_data().columns
cat_cols=X_train.drop(num_cols,axis=1).columns


encoders=[TargetEncoder() for _ in range(len(cat_cols))]


def fit_encode(data,target,encoder):
  nonnulls=np.array(data.dropna())
  target=target.loc[data.dropna().index]
  reshaped=nonnulls.reshape(-1,1)
  encoded=encoder.fit_transform(reshaped,target)
  data.loc[data.notnull()]=np.squeeze(encoded)
  return data

def encode_only(data,encoder):
  nonnulls=np.array(data.dropna())
  reshaped=nonnulls.reshape(-1,1)
  encoded=encoder.transform(reshaped)
  data.loc[data.notnull()]=np.squeeze(encoded)
  return data
  
for i,columns in enumerate(cat_cols):
  fit_encode(X_train[columns],y_train,encoders[i])

for i,columns in enumerate(cat_cols):
  encode_only(X_test[columns],encoders[i])


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.6/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/usr/local/lib/python3.6/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/usr/local/lib/python3.6/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/usr/local/lib/python3.6/dist-packages/category_encoders/utils.py:21: FutureWarning: i

In [11]:
from fancyimpute import KNN

imputer=KNN()
X_train=pd.DataFrame(np.round(imputer.fit_transform(X_train)),columns=X_train.columns)
X_test=pd.DataFrame(np.round(imputer.fit_transform(X_test)),columns=X_test.columns)

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)


Imputing row 1/1095 with 44 missing, elapsed time: 0.631
Imputing row 101/1095 with 0 missing, elapsed time: 0.651
Imputing row 201/1095 with 0 missing, elapsed time: 0.676
Imputing row 301/1095 with 44 missing, elapsed time: 0.696
Imputing row 401/1095 with 0 missing, elapsed time: 0.708
Imputing row 501/1095 with 0 missing, elapsed time: 0.724
Imputing row 601/1095 with 0 missing, elapsed time: 0.745
Imputing row 701/1095 with 0 missing, elapsed time: 0.764
Imputing row 801/1095 with 0 missing, elapsed time: 0.784
Imputing row 901/1095 with 0 missing, elapsed time: 0.804
Imputing row 1001/1095 with 44 missing, elapsed time: 0.822
Imputing row 1/365 with 46 missing, elapsed time: 0.078
Imputing row 101/365 with 0 missing, elapsed time: 0.147
Imputing row 201/365 with 0 missing, elapsed time: 0.190
Imputing row 301/365 with 0 missing, elapsed time: 0.235
